In [1]:
import yfinance as yf
import pandas as pd
import concurrent.futures
from tqdm import tqdm

# Load Russell 2000 tickers from a CSV file
russell_2000_file = 'russell_2000.csv'
russell_2000_df = pd.read_csv(russell_2000_file)

# Create an empty dataframe to store the results
columns = ['Ticker', 'Company', 'EV/EBITDA']
ev_ebitda_df = pd.DataFrame(columns=columns)

# Function to get EV/EBITDA
def get_ev_ebitda(ticker, company):
    try:
        stock = yf.Ticker(ticker)
        enterprise_value = stock.info.get('enterpriseValue', None)
        ebitda = stock.info.get('ebitda', None)
        if enterprise_value is None or ebitda is None or ebitda == 0:
            return {'Ticker': ticker, 'Company': company, 'EV/EBITDA': None}
        ev_ebitda = enterprise_value / ebitda
        return {'Ticker': ticker, 'Company': company, 'EV/EBITDA': ev_ebitda}
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")
        return {'Ticker': ticker, 'Company': company, 'EV/EBITDA': None}

# List to collect data rows
data_rows = []

# Multithreading the fetching process
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(get_ev_ebitda, row['Ticker'], row['Company'])
        for index, row in russell_2000_df.iterrows()
    ]
    
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result['EV/EBITDA'] is not None:
            data_rows.append(result)

# Create DataFrame from the collected data
ev_ebitda_df = pd.DataFrame(data_rows)

# Save the DataFrame to an Excel file
output_file = 'russell_2000_ev_ebitda.xlsx'
ev_ebitda_df.to_excel(output_file, index=False)

print(f"DataFrame has been saved to {output_file}")


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AAC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AAC&crumb=vrsY67BYNzn
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=vrsY67BYNzn
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ACHN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ACHN&crumb=vrsY67BYNzn
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AEL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.c

DataFrame has been saved to russell_2000_ev_ebitda.xlsx
